<a href="https://colab.research.google.com/github/ambatukamza007/yessirr/blob/main/Progect_gis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ตัวสำรองเผื่อมีอะไรผิดพลาดใน code หลัก)
# ✅ ติดตั้งไลบรารีที่จำเป็น (สำหรับ Colab)
!pip install geemap geopandas osmnx google-api-python-client google-auth rasterio scipy --quiet

import ee
import geemap
import os
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Polygon
from geopy.geocoders import Nominatim
from googleapiclient.discovery import build
from google.oauth2 import service_account
import shutil
import json
from google.colab import drive
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds
import numpy as np
from scipy.ndimage import distance_transform_edt

# ✅ Mount Google Drive
print("\U0001F517 Connecting to Google Drive...")
drive.mount('/content/drive')

# ✅ ตั้งค่า path ไปยังไฟล์ JSON บน Drive
KEY_FILE = '/content/drive/MyDrive/EarthEngineExports/srisetthanil71-414b399c88d0.json'

# ✅ โหลด Service Account JSON และกำหนดอีเมล
with open(KEY_FILE) as f:
    sa_info = json.load(f)
SERVICE_ACCOUNT = sa_info["client_email"]

# ✅ ใช้ Service Account Login (GEE + Drive API)
print("\U0001F510 Authenticating Earth Engine & Drive API...")
credentials = service_account.Credentials.from_service_account_file(
    KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine', 'https://www.googleapis.com/auth/drive']
)
ee.Initialize(credentials)
drive_service = build('drive', 'v3', credentials=credentials)
print(f"✅ Authenticated as: {SERVICE_ACCOUNT}")

# ✅ ฟังก์ชันแยกการแปลงพิกัดเป็นจังหวัด/ประเทศ/อำเภอ
geolocator = Nominatim(user_agent="rocket_apogee_tracker_2025", timeout=10)

def get_province_en(address):
    return (
        address.get('state') or
        address.get('city') or
        address.get('region') or
        address.get('county') or
        'Unknown'
    )

def get_district_en(address):
    return (
        address.get('county') or
        address.get('region') or
        address.get('municipality') or
        address.get('suburb') or
        'Unknown'
    )

def gps_to_province(lat, lon):
    location = geolocator.reverse(f"{lat}, {lon}", language='en')
    if location is None or location.raw is None:
        return {"error": "Location not found."}
    address = location.raw.get('address', {})
    province = get_province_en(address)
    district = get_district_en(address)
    country = address.get('country', 'Unknown')
    return {
        "Country": country,
        "Province": province.replace(" ", "_"),
        "District": district.replace(" ", "_"),
        "Full Address": location.address,
        "Address Dict": address
    }

# ✅ รับ lat/lon จากผู้ใช้
lat = float(input("\U0001F4CD ใส่ละติจูด: "))
lon = float(input("\U0001F4CD ใส่ลองจิจูด: "))
location_info = gps_to_province(lat, lon)
province = location_info["Province"]
country = location_info["Country"]
district = location_info["District"]
address = location_info["Address Dict"]
area_query = f"{province}, {country}"
print("\U0001F30D พื้นที่:", area_query)

# ✅ ตั้งค่าพื้นที่วิเคราะห์
roi = ee.Geometry.Point([lon, lat]).buffer(5000)
Map = geemap.Map(center=(lat, lon), zoom=12)

# ✅ ดาวน์โหลดข้อมูล GEE
lulc = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate("2023-01-01", "2023-12-31").median()
slope = ee.Terrain.slope(ee.Image("USGS/SRTMGL1_003").clip(roi))
wind = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
    .filterDate("2023-01-01", "2023-01-07") \
    .select(['u_component_of_wind_10m', 'v_component_of_wind_10m']) \
    .mean()
wind_speed = wind.expression('sqrt(u*u + v*v)', {
    'u': wind.select('u_component_of_wind_10m'),
    'v': wind.select('v_component_of_wind_10m')
}).rename('wind_speed')

# ✅ Export GEE to Drive
export_folder = 'data_got'
export_tasks = {}
print(f"\U0001F4E3 แชร์โฟลเดอร์ '{export_folder}' ให้: {SERVICE_ACCOUNT}")

def export_to_drive(image, description, scale=30):
    file_name = f"{description}_{province}.tif"
    task = ee.batch.Export.image.toDrive(
        image=image.clip(roi),
        description=description,
        folder=export_folder,
        fileNamePrefix=description + f"_{province}",
        region=roi.bounds(),
        scale=scale,
        crs='EPSG:4326',
        maxPixels=1e10
    )
    task.start()
    export_tasks[description] = file_name
    print(f"\U0001F4E4 Export '{description}' -> {file_name}")

export_to_drive(lulc.select('label'), "LULC", 10)
export_to_drive(slope, "Slope", 30)
export_to_drive(wind_speed, "WindSpeed", 30000)

# ✅ แสดงผลบนแผนที่
Map.addLayer(lulc.select('label'), {}, "LULC")
Map.addLayer(slope, {"min": 0, "max": 60}, "Slope")
Map.addLayer(wind_speed, {"min": 0, "max": 15}, "Wind Speed")
Map.addLayer(roi, {}, "ROI")

# ✅ สร้าง polygon ขอบเขต
output_dir = "/content/outputs"
os.makedirs(output_dir, exist_ok=True)
try:
    province_boundary = ox.geocode_to_gdf(area_query)
    if not province_boundary.empty:
        province_boundary.to_file(f"{output_dir}/province_boundary.shp")
        province_boundary.to_file(f"{output_dir}/province_boundary.geojson", driver="GeoJSON")
        Map.add_gdf(province_boundary, layer_name="Province Boundary")
except:
    print("❌ โหลดขอบเขตจังหวัดไม่สำเร็จ")

try:
    location = geolocator.geocode(area_query)
    if location and 'boundingbox' in location.raw:
        bbox = [float(x) for x in location.raw['boundingbox']]
        poly = Polygon([
            (bbox[2], bbox[0]), (bbox[2], bbox[1]),
            (bbox[3], bbox[1]), (bbox[3], bbox[0]),
            (bbox[2], bbox[0])
        ])
    else:
        raise ValueError("No boundingbox")
except:
    print("⚠️ ใช้ fallback จาก Earth Engine buffer")
    poly = Polygon(ee.Geometry.Point([lon, lat]).buffer(5000).bounds().coordinates().getInfo()[0])

area_gdf = gpd.GeoDataFrame(geometry=[poly], crs="EPSG:4326")

# ✅ ฟังก์ชัน export & rasterize

def export_gdf(gdf, name):
    if not gdf.empty:
        gdf.to_file(f"{output_dir}/{name}.geojson", driver="GeoJSON")
        gdf.to_file(f"{output_dir}/{name}.shp", driver="ESRI Shapefile", encoding="utf-8")

def rasterize_gdf(gdf, output_tif, bounds, width=512, height=512):
    if gdf.empty:
        return
    transform = from_bounds(*bounds, width=width, height=height)
    shapes = [(geom, 1) for geom in gdf.geometry]
    mask = rasterize(shapes=shapes, out_shape=(height, width), transform=transform, fill=0)
    with rasterio.open(output_tif, 'w', driver='GTiff', height=mask.shape[0], width=mask.shape[1], count=1, dtype=mask.dtype, crs="EPSG:4326", transform=transform) as dst:
        dst.write(mask, 1)

def rasterize_proximity(gdf, output_tif, bounds, width=512, height=512):
    if gdf.empty:
        return
    transform = from_bounds(*bounds, width=width, height=height)
    shapes = [(geom, 1) for geom in gdf.geometry]
    mask = rasterize(shapes=shapes, out_shape=(height, width), transform=transform, fill=0)
    distance = distance_transform_edt(1 - mask)
    with rasterio.open(output_tif, 'w', driver='GTiff', height=distance.shape[0], width=distance.shape[1], count=1, dtype=distance.dtype, crs="EPSG:4326", transform=transform) as dst:
        dst.write(distance, 1)

# ✅ ดึงถนน + proximity
try:
    roads = ox.graph_from_polygon(poly.buffer(0.001), network_type='drive')
    roads_gdf = ox.graph_to_gdfs(roads, nodes=False, edges=True)
    Map.add_gdf(roads_gdf, layer_name="Roads")
    export_gdf(roads_gdf, "roads")
    rasterize_gdf(roads_gdf, f"{output_dir}/roads_raster.tif", poly.bounds)
    rasterize_proximity(roads_gdf, f"{output_dir}/roads_proximity.tif", poly.bounds)
except:
    print("❌ ถนนไม่พร้อมใช้งาน")

# ✅ ดึงสายไฟ
try:
    power_lines = ox.features_from_polygon(poly.buffer(0.001), tags={'power': 'line'})
    power_lines = power_lines[power_lines.geometry.notnull()]
    if not power_lines.empty:
        Map.add_gdf(power_lines, layer_name="Power Lines")
        export_gdf(power_lines, "power_lines")
        rasterize_gdf(power_lines, f"{output_dir}/powerlines_raster.tif", poly.bounds)
        rasterize_proximity(power_lines, f"{output_dir}/powerlines_proximity.tif", poly.bounds)
except:
    print("❌ สายไฟไม่พร้อมใช้งาน")

# ✅ ดึงโรงไฟฟ้า
try:
    power_plants = ox.features_from_polygon(poly.buffer(0.001), tags={'power': 'plant'})
    power_plants = power_plants[power_plants.geometry.notnull()]
    if not power_plants.empty:
        Map.add_gdf(power_plants, layer_name="Power Plants")
        export_gdf(power_plants, "power_plants")
        rasterize_gdf(power_plants, f"{output_dir}/powerplants_raster.tif", poly.bounds)
        rasterize_proximity(power_plants, f"{output_dir}/powerplants_proximity.tif", poly.bounds)
except:
    print("❌ ดึงข้อมูลโรงไฟฟ้าไม่สำเร็จ")

# ✅ สร้าง ZIP และดาวน์โหลด
shutil.make_archive(output_dir, 'zip', output_dir)
from google.colab import files
files.download(f"{output_dir}.zip")

# ✅ แสดงแผนที่
Map


In [ ]:
# ✅ ติดตั้งไลบรารีที่จำเป็น (สำหรับ Colab)
!pip install geemap geopandas osmnx google-api-python-client google-auth rasterio scipy --quiet

import ee
import geemap
import os
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Polygon
from geopy.geocoders import Nominatim
from googleapiclient.discovery import build
from google.oauth2 import service_account
import shutil
import json
from google.colab import drive
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds
import numpy as np
from scipy.ndimage import distance_transform_edt

# ✅ Mount Google Drive
print("\U0001F517 Connecting to Google Drive...")
drive.mount('/content/drive')

# ✅ ตั้งค่า path ไปยังไฟล์ JSON บน Drive
KEY_FILE = '/content/drive/MyDrive/EarthEngineExports/srisetthanil71-414b399c88d0.json'

# ✅ โหลด Service Account JSON และกำหนดอีเมล
with open(KEY_FILE) as f:
    sa_info = json.load(f)
SERVICE_ACCOUNT = sa_info["client_email"]

# ✅ ใช้ Service Account Login (GEE + Drive API)
print("\U0001F510 Authenticating Earth Engine & Drive API...")
credentials = service_account.Credentials.from_service_account_file(
    KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine', 'https://www.googleapis.com/auth/drive']
)
ee.Initialize(credentials)
drive_service = build('drive', 'v3', credentials=credentials)
print(f"✅ Authenticated as: {SERVICE_ACCOUNT}")

# ✅ ฟังก์ชันแยกการแปลงพิกัดเป็นจังหวัด/ประเทศ/อำเภอ
geolocator = Nominatim(user_agent="rocket_apogee_tracker_2025", timeout=10)

def get_province_en(address):
    return (
        address.get('state') or
        address.get('city') or
        address.get('region') or
        address.get('county') or
        'Unknown'
    )

def get_district_en(address):
    return (
        address.get('county') or
        address.get('region') or
        address.get('municipality') or
        address.get('suburb') or
        'Unknown'
    )

def gps_to_province(lat, lon):
    location = geolocator.reverse(f"{lat}, {lon}", language='en')
    if location is None or location.raw is None:
        return {"error": "Location not found."}
    address = location.raw.get('address', {})
    province = get_province_en(address)
    district = get_district_en(address)
    country = address.get('country', 'Unknown')
    return {
        "Country": country,
        "Province": province.replace(" ", "_"),
        "District": district.replace(" ", "_"),
        "Full Address": location.address,
        "Address Dict": address
    }

# ✅ รับ lat/lon จากผู้ใช้
lat = float(input("\U0001F4CD ใส่ละติจูด: "))
lon = float(input("\U0001F4CD ใส่ลองจิจูด: "))
location_info = gps_to_province(lat, lon)
province = location_info["Province"]
country = location_info["Country"]
district = location_info["District"]
address = location_info["Address Dict"]
area_query = f"{province}, {country}"
print("\U0001F30D พื้นที่:", area_query)

# ✅ ตั้งค่าพื้นที่วิเคราะห์
roi = ee.Geometry.Point([lon, lat]).buffer(5000)
Map = geemap.Map(center=(lat, lon), zoom=12)

# ✅ ดาวน์โหลดข้อมูล GEE
lulc = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate("2023-01-01", "2023-12-31").median()
slope = ee.Terrain.slope(ee.Image("USGS/SRTMGL1_003").clip(roi))
wind = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
    .filterDate("2023-01-01", "2023-01-07") \
    .select(['u_component_of_wind_10m', 'v_component_of_wind_10m']) \
    .mean()
wind_speed = wind.expression('sqrt(u*u + v*v)', {
    'u': wind.select('u_component_of_wind_10m'),
    'v': wind.select('v_component_of_wind_10m')
}).rename('wind_speed')

# ✅ Export GEE to Drive
export_folder = 'data_got'
export_tasks = {}
print(f"\U0001F4E3 แชร์โฟลเดอร์ '{export_folder}' ให้: {SERVICE_ACCOUNT}")

def export_to_drive(image, description, scale=30):
    file_name = f"{description}_{province}.tif"
    task = ee.batch.Export.image.toDrive(
        image=image.clip(roi),
        description=description,
        folder=export_folder,
        fileNamePrefix=description + f"_{province}",
        region=roi.bounds(),
        scale=scale,
        crs='EPSG:4326',
        maxPixels=1e10
    )
    task.start()
    export_tasks[description] = file_name
    print(f"\U0001F4E4 Export '{description}' -> {file_name}")

export_to_drive(lulc.select('label'), "LULC", 10)
export_to_drive(slope, "Slope", 30)
export_to_drive(wind_speed, "WindSpeed", 30000)

# ✅ แสดงผลบนแผนที่
Map.addLayer(lulc.select('label'), {}, "LULC")
Map.addLayer(slope, {"min": 0, "max": 60}, "Slope")
Map.addLayer(wind_speed, {"min": 0, "max": 15}, "Wind Speed")
Map.addLayer(roi, {}, "ROI")

# ✅ สร้าง polygon ขอบเขต
output_dir = "/content/outputs"
os.makedirs(output_dir, exist_ok=True)

# ✅ ลบไฟล์ที่ไม่ใช่ชุด proximity/raster ปัจจุบัน
for file in os.listdir(output_dir):
    if not any(key in file for key in ["roads_", "powerlines_", "power_plants_", "province_boundary_"]):
        os.remove(os.path.join(output_dir, file))

try:
    province_boundary = ox.geocode_to_gdf(area_query)
    if not province_boundary.empty:
        province_boundary.to_file(f"{output_dir}/province_boundary.shp")
        province_boundary.to_file(f"{output_dir}/province_boundary.geojson", driver="GeoJSON")
        Map.add_gdf(province_boundary, layer_name="Province Boundary")
except:
    print("❌ โหลดขอบเขตจังหวัดไม่สำเร็จ")

try:
    location = geolocator.geocode(area_query)
    if location and 'boundingbox' in location.raw:
        bbox = [float(x) for x in location.raw['boundingbox']]
        poly = Polygon([
            (bbox[2], bbox[0]), (bbox[2], bbox[1]),
            (bbox[3], bbox[1]), (bbox[3], bbox[0]),
            (bbox[2], bbox[0])
        ])
    else:
        raise ValueError("No boundingbox")
except:
    print("⚠️ ใช้ fallback จาก Earth Engine buffer")
    poly = Polygon(ee.Geometry.Point([lon, lat]).buffer(5000).bounds().coordinates().getInfo()[0])

area_gdf = gpd.GeoDataFrame(geometry=[poly], crs="EPSG:4326")

# ✅ ฟังก์ชัน export & rasterize

def export_gdf(gdf, name):
    if not gdf.empty:
        gdf.to_file(f"{output_dir}/{name}.geojson", driver="GeoJSON")
        gdf.to_file(f"{output_dir}/{name}.shp", driver="ESRI Shapefile", encoding="utf-8")

def rasterize_gdf(gdf, output_tif, bounds, width=512, height=512):
    if gdf.empty:
        return
    transform = from_bounds(*bounds, width=width, height=height)
    shapes = [(geom, 1) for geom in gdf.geometry]
    mask = rasterize(shapes=shapes, out_shape=(height, width), transform=transform, fill=0)
    with rasterio.open(output_tif, 'w', driver='GTiff', height=mask.shape[0], width=mask.shape[1], count=1, dtype=mask.dtype, crs="EPSG:4326", transform=transform) as dst:
        dst.write(mask, 1)

def rasterize_proximity(gdf, output_tif, bounds, width=512, height=512):
    if gdf.empty:
        return
    transform = from_bounds(*bounds, width=width, height=height)
    shapes = [(geom, 1) for geom in gdf.geometry]
    mask = rasterize(shapes=shapes, out_shape=(height, width), transform=transform, fill=0)
    distance = distance_transform_edt(1 - mask)
    norm_distance = distance / np.max(distance) if np.max(distance) > 0 else distance
    with rasterio.open(output_tif, 'w', driver='GTiff', height=distance.shape[0], width=distance.shape[1], count=1, dtype=norm_distance.dtype, crs="EPSG:4326", transform=transform) as dst:
        dst.write(norm_distance, 1)

# ✅ ดึงถนน + proximity
try:
    roads = ox.graph_from_polygon(poly.buffer(0.001), network_type='drive')
    roads_gdf = ox.graph_to_gdfs(roads, nodes=False, edges=True)
    Map.add_gdf(roads_gdf, layer_name="Roads")
    export_gdf(roads_gdf, "roads")
    rasterize_gdf(roads_gdf, f"{output_dir}/roads_raster.tif", poly.bounds)
    rasterize_proximity(roads_gdf, f"{output_dir}/roads_proximity.tif", poly.bounds)
except:
    print("❌ ถนนไม่พร้อมใช้งาน")

try:
    power_lines = ox.features_from_polygon(poly.buffer(0.001), tags={'power': 'line'})
    power_lines = power_lines[power_lines.geometry.notnull()]
    if not power_lines.empty:
        Map.add_gdf(power_lines, layer_name="Power Lines")
        export_gdf(power_lines, "power_lines")
        rasterize_gdf(power_lines, f"{output_dir}/powerlines_raster.tif", poly.bounds)
        rasterize_proximity(power_lines, f"{output_dir}/powerlines_proximity.tif", poly.bounds)
except:
    print("❌ สายไฟไม่พร้อมใช้งาน")

try:
    power_plants = ox.features_from_polygon(poly.buffer(0.001), tags={'power': 'plant'})
    power_plants = power_plants[power_plants.geometry.notnull()]
    if not power_plants.empty:
        Map.add_gdf(power_plants, layer_name="Power Plants")
        export_gdf(power_plants, "power_plants")
        rasterize_gdf(power_plants, f"{output_dir}/powerplants_raster.tif", poly.bounds)
        rasterize_proximity(power_plants, f"{output_dir}/powerplants_proximity.tif", poly.bounds)
except:
    print("❌ ดึงข้อมูลโรงไฟฟ้าไม่สำเร็จ")

# ✅ สร้าง ZIP และดาวน์โหลด
shutil.make_archive(output_dir, 'zip', output_dir)
from google.colab import files
files.download(f"{output_dir}.zip")

# ✅ แสดงแผนที่
Map


In [17]:
# ✅ ติดตั้งไลบรารีที่จำเป็น (สำหรับ Colab)
!pip install geemap geopandas osmnx google-api-python-client google-auth rasterio scipy --quiet

import ee
import geemap
import os
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Polygon
from geopy.geocoders import Nominatim
from googleapiclient.discovery import build
from google.oauth2 import service_account
import shutil
import json
from google.colab import drive
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds
import numpy as np
from scipy.ndimage import distance_transform_edt

# ✅ Mount Google Drive
print("\U0001F517 Connecting to Google Drive...")
drive.mount('/content/drive')

# ✅ ตั้งค่า path ไปยังไฟล์ JSON บน Drive
KEY_FILE = '/content/drive/MyDrive/EarthEngineExports/srisetthanil71-414b399c88d0.json'

# ✅ โหลด Service Account JSON และกำหนดอีเมล
with open(KEY_FILE) as f:
    sa_info = json.load(f)
SERVICE_ACCOUNT = sa_info["client_email"]

# ✅ ใช้ Service Account Login (GEE + Drive API)
print("\U0001F510 Authenticating Earth Engine & Drive API...")
credentials = service_account.Credentials.from_service_account_file(
    KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine', 'https://www.googleapis.com/auth/drive']
)
ee.Initialize(credentials)
drive_service = build('drive', 'v3', credentials=credentials)
print(f"✅ Authenticated as: {SERVICE_ACCOUNT}")

# ✅ ฟังก์ชันแยกการแปลงพิกัดเป็นจังหวัด/ประเทศ/อำเภอ
geolocator = Nominatim(user_agent="rocket_apogee_tracker_2025", timeout=10)

def get_province_en(address):
    return (
        address.get('state') or
        address.get('city') or
        address.get('region') or
        address.get('county') or
        'Unknown'
    )

def get_district_en(address):
    return (
        address.get('county') or
        address.get('region') or
        address.get('municipality') or
        address.get('suburb') or
        'Unknown'
    )

def gps_to_province(lat, lon):
    location = geolocator.reverse(f"{lat}, {lon}", language='en')
    if location is None or location.raw is None:
        return {"error": "Location not found."}
    address = location.raw.get('address', {})
    province = get_province_en(address)
    district = get_district_en(address)
    country = address.get('country', 'Unknown')
    return {
        "Country": country,
        "Province": province.replace(" ", "_"),
        "District": district.replace(" ", "_"),
        "Full Address": location.address,
        "Address Dict": address
    }

# ✅ รับ lat/lon จากผู้ใช้
lat = float(input("\U0001F4CD ใส่ละติจูด: "))
lon = float(input("\U0001F4CD ใส่ลองจิจูด: "))
location_info = gps_to_province(lat, lon)
province = location_info["Province"]
country = location_info["Country"]
district = location_info["District"]
address = location_info["Address Dict"]
area_query = f"{province}, {country}"
print("\U0001F30D พื้นที่:", area_query)

# ✅ ตั้งค่าพื้นที่วิเคราะห์
roi = ee.Geometry.Point([lon, lat]).buffer(5000)
Map = geemap.Map(center=(lat, lon), zoom=12)

# ✅ ดาวน์โหลดข้อมูล GEE
lulc = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate("2023-01-01", "2023-12-31").median()
slope = ee.Terrain.slope(ee.Image("USGS/SRTMGL1_003").clip(roi))
wind = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
    .filterDate("2023-01-01", "2023-01-07") \
    .select(['u_component_of_wind_10m', 'v_component_of_wind_10m']) \
    .mean()
wind_speed = wind.expression('sqrt(u*u + v*v)', {
    'u': wind.select('u_component_of_wind_10m'),
    'v': wind.select('v_component_of_wind_10m')
}).rename('wind_speed')

# ✅ Export GEE to Drive
export_folder = 'data_got'
export_tasks = {}
print(f"\U0001F4E3 แชร์โฟลเดอร์ '{export_folder}' ให้: {SERVICE_ACCOUNT}")

def export_to_drive(image, description, scale=30):
    file_name = f"{description}_{province}.tif"
    task = ee.batch.Export.image.toDrive(
        image=image.clip(roi),
        description=description,
        folder=export_folder,
        fileNamePrefix=description + f"_{province}",
        region=roi.bounds(),
        scale=scale,
        crs='EPSG:4326',
        maxPixels=1e10
    )
    task.start()
    export_tasks[description] = file_name
    print(f"\U0001F4E4 Export '{description}' -> {file_name}")

export_to_drive(lulc.select('label'), "LULC", 10)
export_to_drive(slope, "Slope", 30)
export_to_drive(wind_speed, "WindSpeed", 30000)

# ✅ แสดงผลบนแผนที่
Map.addLayer(lulc.select('label'), {}, "LULC")
Map.addLayer(slope, {"min": 0, "max": 60}, "Slope")
Map.addLayer(wind_speed, {"min": 0, "max": 15}, "Wind Speed")
Map.addLayer(roi, {}, "ROI")

# ✅ สร้าง polygon ขอบเขต
output_dir = "/content/outputs"
os.makedirs(output_dir, exist_ok=True)

# ✅ โหลดขอบเขตจังหวัด
try:
    province_boundary = ox.geocode_to_gdf(area_query)
    if not province_boundary.empty:
        province_boundary.to_file(f"{output_dir}/province_boundary.shp")
        province_boundary.to_file(f"{output_dir}/province_boundary.geojson", driver="GeoJSON")
        Map.add_gdf(province_boundary, layer_name="Province Boundary")
except:
    print("❌ โหลดขอบเขตจังหวัดไม่สำเร็จ")

# ✅ โหลดขอบเขตอำเภอ
try:
    district_query = f"{district}, {province}, {country}"
    district_boundary = ox.geocode_to_gdf(district_query)
    if not district_boundary.empty:
        district_boundary.to_file(f"{output_dir}/district_boundary.shp")
        district_boundary.to_file(f"{output_dir}/district_boundary.geojson", driver="GeoJSON")
        Map.add_gdf(district_boundary, layer_name="District Boundary")
except:
    print("❌ โหลดขอบเขตอำเภอไม่สำเร็จ")

# ✅ แสดงแผนที่
Map


🔗 Connecting to Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔐 Authenticating Earth Engine & Drive API...
✅ Authenticated as: iloveiloveher@srisetthanil71.iam.gserviceaccount.com
📍 ใส่ละติจูด: 17.1740
📍 ใส่ลองจิจูด: 102.4495
🌍 พื้นที่: Mueang_Nong_Bua_Lam_Phu, Thailand
📣 แชร์โฟลเดอร์ 'data_got' ให้: iloveiloveher@srisetthanil71.iam.gserviceaccount.com
📤 Export 'LULC' -> LULC_Mueang_Nong_Bua_Lam_Phu.tif
📤 Export 'Slope' -> Slope_Mueang_Nong_Bua_Lam_Phu.tif
📤 Export 'WindSpeed' -> WindSpeed_Mueang_Nong_Bua_Lam_Phu.tif
🔍 Querying district boundary: Mueang_Nong_Bua_Lam_Phu, Mueang_Nong_Bua_Lam_Phu, Thailand
⚠️ ไม่พบขอบเขตอำเภอจาก OSM - ใช้ fallback buffer แทน


Map(center=[17.174, 102.4495], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [ ]:
# ✅ ใช้ sensor (อุณหภูมิ & ความกดอากาศ) มาร่วมกับ DEM + ERA5 ลม และ export เป็น GeoTIFF (ระดับจังหวัด + ระดับอำเภอ)

import ee
import geemap
import datetime
import osmnx as ox

# ✅ ดึงค่าจากโค้ดก่อนหน้า: lat, lon, slope, wind ต้องรันมาแล้ว

# ✅ สร้าง roi จังหวัด
province_boundary = ox.geocode_to_gdf(f"{province}, {country}")
roi_province = geemap.geopandas_to_ee(province_boundary)

# ✅ สร้าง roi อำเภอ (district)
district_boundary = ox.geocode_to_gdf(f"{district}, {province}, {country}")
roi_district = geemap.geopandas_to_ee(district_boundary)

# ✅ แยก component ลม
u = wind.select('u_component_of_wind_10m')
v = wind.select('v_component_of_wind_10m')
wind_speed = wind.expression('sqrt(u*u + v*v)', {'u': u, 'v': v})

# ✅ จุด sensor เดียว (จาก LoRa จริง)
sensor_points = [
    {'lat': lat, 'lon': lon, 'P': P, 'T': T}  # ดึงจากโค้ดหลัก
]
sensor_point = sensor_points[0]
P_sensor = sensor_point['P']
T_sensor = sensor_point['T']
sensor_geom = ee.Geometry.Point([sensor_point['lon'], sensor_point['lat']])

# ✅ ดึง ERA5 ล่าสุด (~6 วันก่อน)
today = datetime.datetime.utcnow().date()
latest_date = ee.Date(str(today)).advance(-6, 'day')
start_date = latest_date.format('YYYY-MM-dd')
end_date = latest_date.advance(1, 'day').format('YYYY-MM-dd')

era5_pressure = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
    .filterDate(start_date, end_date) \
    .select("surface_pressure") \
    .mean() \
    .divide(100)

era5_at_sensor = era5_pressure.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=sensor_geom,
    scale=1000
).get("surface_pressure")

correction_factor = ee.Number(P_sensor).divide(era5_at_sensor)
pressure_surface = era5_pressure.multiply(correction_factor)

adjusted_u = u.multiply(pressure_surface.divide(1013.25))
adjusted_v = v.multiply(pressure_surface.divide(1013.25))
adjusted_wind = adjusted_u.pow(2).add(adjusted_v.pow(2)).sqrt()

weighted_wind = adjusted_wind.multiply(slope.divide(90).add(1))

R = 287.05
rho = P_sensor * 100 / (R * (T_sensor + 273.15))
wind_energy = adjusted_wind.pow(3).multiply(0.5 * rho)

Map.addLayer(weighted_wind.clip(roi_province), {'min': 0, 'max': 15}, "Weighted Wind - Province")
Map.addLayer(wind_energy.clip(roi_province), {'min': 0, 'max': 500}, "Wind Energy - Province")
Map.addLayer(weighted_wind.clip(roi_district), {'min': 0, 'max': 15}, "Weighted Wind - District")
Map.addLayer(wind_energy.clip(roi_district), {'min': 0, 'max': 500}, "Wind Energy - District")
Map

# ✅ Export ทั้งระดับจังหวัด
for name, roi in [("Province", roi_province), ("District", roi_district)]:
    wind_file = f"WeightedWindSensor_{province}_{name}.tif"
    energy_file = f"WindEnergy_{province}_{name}.tif"

    task = ee.batch.Export.image.toDrive(
        image=weighted_wind.clip(roi),
        description=f"WeightedWind_{name}",
        folder='data_got',
        fileNamePrefix=wind_file,
        region=roi.bounds(),
        scale=300,
        crs='EPSG:4326',
        maxPixels=1e10
    )
    task.start()
    print(f"📤 Exporting {wind_file} to Google Drive")

    task_energy = ee.batch.Export.image.toDrive(
        image=wind_energy.clip(roi),
        description=f"WindEnergy_{name}",
        folder='data_got',
        fileNamePrefix=energy_file,
        region=roi.bounds(),
        scale=300,
        crs='EPSG:4326',
        maxPixels=1e10
    )
    task_energy.start()
    print(f"⚡ Exporting {energy_file} to Google Drive")
